In [25]:
import re
import json

In [26]:
FILENAME='kiseki'

qrc_line_re=re.compile(r'^\[(\d+),(\d+)\](.*)$')
qrc_chunk_re=re.compile(r'^(.*)\((\d+),(\d+)$')

In [27]:
def parse_qrc(data):
    INF=2147483647

    out=[]
    line_time=[]

    def apply_chunk(data,time_s,dt):
        if time_s is not None:
            chunk_action.append([time_s,True,chunkid])
            chunk_action.append([time_s+dt,False,chunkid])

    for line_s in data.split('\n'):
        line=qrc_line_re.match(line_s)
        if not line:
            print('ignored LINE:',line_s)
            continue

        time_s,dt,content=line.groups()
        time_s=int(time_s)
        dt=int(dt)
        
        line_time.append(time_s)
        cur_line=[] # list of [time,txt]

        splited_content=content.split(')')
        for ind,chunk_s in enumerate(splited_content):
            chunk=qrc_chunk_re.match(chunk_s)
            if not chunk:
                if len(splited_content)==ind+1: # last chunk without timestamp
                    if chunk_s:
                        cur_line.append([INF,chunk_s])
                else: # normal ')'
                    cur_line[-1][1]+=chunk_s+')'
                    #splited_content[ind+1]=chunk_s+')'+splited_content[ind+1]
                continue

            content,time_s,dt=chunk.groups()
            time_s=int(time_s)
            #dt=int(dt)
            cur_line.append([time_s,content])
            
        out.append(cur_line)

    return {
        'lines': out,
        'line_time': line_time,
    }

In [28]:
res_orig=parse_qrc(open(FILENAME+'_orig.txt',encoding='utf-8').read())
res_roma=parse_qrc(open(FILENAME+'_roma.txt',encoding='utf-8').read())
res_trans=parse_qrc(open(FILENAME+'_trans.txt',encoding='utf-8').read())

ignored LINE: [ti:軌跡 (轨迹)]
ignored LINE: [ar:Roselia]
ignored LINE: [al:Opera of the wasteland]
ignored LINE: [by:]
ignored LINE: [offset:0]
ignored LINE: [kana:1き1せ(957,136)き(1093,96)111し1おり1た1き(3197,103)ょ(3301,103)く(3404,185)1ふじ1えい1りゅう1た1ろう1くつ1ひ(23951,232)も(24183,285)1ほど1むす1な(26999,248)お(27247,193)1わか1と1ぎ1つな1ため1で1あ1ひと1あゆ1かな1むね1なか1おぼ1まぶた1と1むか1 か1わら2あなた1ひとみ1き1れい1めぐ1あ1いのち1く1かえ1はじ1な1みち1ふ1む1まえ1み1めぐ1ち1きゅう2あなた1わたし1すす1にぎ1て1は な1お1きずな2あした1く1あ1まえ1ふ1ま1うたが1ほ1しょう1ひ1び1はかな1ひと1だ1ふ1かえ1やさ1み1まい1にち1たい1せつ1のこ1やわ1ここ1ち1いろ1き1れい1うた1いと1つ1あふ1だ1おも1ほし1またた1めぐ1ち1きゅう2あなた1わたし1すす1め1ざ1ば1しょ1ちが1か1け1しき1よみがえ1よみがえ1すがた1すがた1こころ1だ1つら1だれ1うそ1さと1たば1こと1ば1つた1こ1こ1あ2あなた1わたし1き1せき1ひと1わす1あつ1めぐ1ち1きゅう2あなた1わたし1すす1にぎ1て1はな1お1きずな1いく1せん1えい1えん1かさ]
ignored LINE: 
ignored LINE: [ti:軌跡 (轨迹)]
ignored LINE: [ar:Roselia]
ignored LINE: [al:Opera of the wasteland]
ignored LINE: [by:]
ignored LINE: [offset:0]
ignored LINE: 


In [29]:
assert len(res_orig['lines'])==len(res_roma['lines'])==len(res_trans['lines'])
L=len(res_orig['lines'])

In [30]:
res={
    'lines': [{
        'orig': res_orig['lines'][l],
        'roma': res_roma['lines'][l],
        'trans': res_trans['lines'][l],
    } for l in range(L)],
    'time': [[t,None] for t in res_orig['line_time']],
    'default_line': {
        'orig': [[0,'軌跡 - Roselia']],
        'roma': [],
        'trans': [[0,'轨迹 - Roselia']],
    },
}

In [31]:
with open('res.json','w') as f:
    json.dump(res,f)